In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45400
2,Huelva,Confirmados PDIA 14 días,300
3,Huelva,Tasa PDIA 14 días,"58,46015940136797"
4,Huelva,Confirmados PDIA 7 días,117
5,Huelva,Tasa PDIA 7 dias,"22,79946216653351"
6,Huelva,Total Confirmados,45603
7,Huelva,Curados,44417
8,Huelva,Fallecidos,415


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45400.0


/tmp/ipykernel_4726/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12541.0


/tmp/ipykernel_4726/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4726/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4726/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4726/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6254 personas en los últimos 7 días 

Un positivo PCR cada 2179 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45400.0,117.0,300.0,513170.0,22.799462,58.460159,30.0
Huelva-Costa,26879.0,71.0,178.0,289548.0,24.520978,61.475127,18.0
Condado-Campiña,14093.0,36.0,105.0,156231.0,23.042802,67.208173,11.0
Gibraleón,942.0,11.0,19.0,12737.0,86.362566,149.171704,6.0
Lepe,3014.0,10.0,20.0,27880.0,35.868006,71.736011,6.0
Ayamonte,2075.0,8.0,25.0,21104.0,37.907506,118.460955,4.0
Cartaya,1978.0,12.0,28.0,20083.0,59.752029,139.421401,4.0
Huelva (capital),12541.0,23.0,66.0,143837.0,15.990322,45.885273,4.0
Moguer,1958.0,7.0,15.0,21867.0,32.011707,68.596515,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,22.0,2.0,2.0,713.0,280.504909,280.504909,0.0
Rosal de la Frontera,74.0,3.0,3.0,1697.0,176.782557,176.782557,0.0
Gibraleón,942.0,11.0,19.0,12737.0,86.362566,149.171704,6.0
Palos de la Frontera,1030.0,3.0,17.0,11742.0,25.549310,144.779424,0.0
Cartaya,1978.0,12.0,28.0,20083.0,59.752029,139.421401,4.0
Bollullos Par del Condado,1282.0,4.0,20.0,14387.0,27.802878,139.014388,0.0
Ayamonte,2075.0,8.0,25.0,21104.0,37.907506,118.460955,4.0
Paterna del Campo,208.0,1.0,3.0,3457.0,28.926815,86.780445,NaN
Almonte,2286.0,5.0,20.0,24507.0,20.402334,81.609336,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Palos de la Frontera,1030.0,3.0,17.0,11742.0,25.549310,144.779424,0.0,0.176471
Bollullos Par del Condado,1282.0,4.0,20.0,14387.0,27.802878,139.014388,0.0,0.200000
Punta Umbría,1264.0,2.0,8.0,15355.0,13.025073,52.100293,0.0,0.250000
Almonte,2286.0,5.0,20.0,24507.0,20.402334,81.609336,1.0,0.250000
Ayamonte,2075.0,8.0,25.0,21104.0,37.907506,118.460955,4.0,0.320000
Paterna del Campo,208.0,1.0,3.0,3457.0,28.926815,86.780445,0.0,0.333333
Palma del Condado (La),1417.0,1.0,3.0,10801.0,9.258402,27.775206,0.0,0.333333
Condado-Campiña,14093.0,36.0,105.0,156231.0,23.042802,67.208173,11.0,0.342857
Huelva (capital),12541.0,23.0,66.0,143837.0,15.990322,45.885273,4.0,0.348485
